In [ ]:
import csv

# Global Customer list
customers = []

In [ ]:
# Classes
# Account class
class Account:
    """
    Base account class used by Checking, Savings, and Credit.

    Attributes:
    account_id: four-digit account ID as string
    balance: current balance of the account (for credit accounts shows amount owed) as float
    interest: rate applies to account as float

    """
    def __init__(self, account_id = "0000", balance = 0.0, interest = 0):
        """
        Initialize a general account

        """
        self.account_id = account_id
        self.balance = balance
        self.interest = interest

    def __str__(self):
        """
        Print account data
        """
        return f"ID: {self.account_id}, Balance: ${self.balance:.2f}, Interest: {self.interest*100}%"
    
    # Get and Set methods for account ID

    def get_account_id(self):
        """
        Return the account ID

        """
        #print("getting account ID")
        return self._account_id
    
    def set_account_id(self,value):
        """
        Set the account ID
        """
        #print("setting account id to", value)

        if not isinstance(value,str) or len(value) != 4:
            raise ValueError("Account ID must be a 4-digit string")
        
        for char in value:
            if char not in "0123456789":
                raise ValueError("Account ID must contail numeric digits")
        
        self._account_id = value
    
    account_id = property(get_account_id, set_account_id)

    # Get and Set methods for balance
    
    def get_balance(self):
        """
        Return the current balance

        """
        #print("getting balance")
        return self._balance
    
    def set_balance(self,value):
        """
        Set the account balance

        """
        #print("setting balance to", value)

        if not isinstance(value,(int,float)):
            raise TypeError("Balance must be a number")
        
        self._balance = float(value)

    balance = property(get_balance, set_balance)

    # Core methods

    def deposit(self,amount):
        """
        Add money to the account

        """
        self.balance += amount

    def withdraw(self,amount):
        """
        Remove money from the account if there is enough funds

        """
        if amount > self.balance:
            print("Withdrawal denied: insufficient funds")
            return False
        self.balance -= amount
        return True
    
# Checking Class
class Checking(Account):
    """
    Checking account class
    Interest rate is always 0%

    """

    def __init__(self,account_id, balance):
        super().__init__(account_id, balance, interest= 0)

# Savings Class
class Savings(Account):
    """
    Savings account class
    Interest rate is always 1%

    """

    def _init__(self,account_id, balance):
        super().__init__(account_id, balance, interest = 0.01)

# Credit Class
class Credit(Account):
    """
    Credit account class. 

    Attributes:
    credit_limit: Maximum amount allowed to be charged

    """

    def __init__(self, account_id, balance, credit_limit):
        super().__init__(account_id, balance, interest= 0.30)
        self.credit_limit = credit_limit

    # Get and Set methods for credit limit

    def get_credit_limit(self):
        """
        Return credit limit

        """

        #print("Getting credit limit.")
        return self._credit_limit
    
    def set_credit_limit(self, value):
        """
        Set the credit limit to value

        """
        #print("Setting credit limit to", value)

        if not isinstance(value,(int,float)):
            raise TypeError("Credit limit must be a number")
        
        self._credit_limit = float(value)

    credit_limit = property(get_credit_limit, set_credit_limit)


    # Credit Methods
    def charge(self, amount):
        """
        Add a charge to the credit balance if it
        does not exceed the credit limit.
        """

        if self.balance + amount > self.credit_limit:
            print("Charge denied: credit limit exceeded")
            return False
        
        self.balance += amount
        return True
    
    def make_payment(self, amount):
        """
        Reduce the credit balance

        """

        if amount > self.balance:
            print("Payment denied: too large")
            return False
        
        self.balance -= amount
        return True

# Customer Class
class Customer:
    """
    Represent one customer with three accounts.

    Attributes;
    username: string representing a person uniquely
    checking: Checking object
    Savings: Savings object
    Credit: Credit object

    """

    def __init__(self, username, checking, savings, credit):
        self.username = username
        self.checking = checking
        self.savings = savings
        self.credit = credit

    def __str__(self):
        """
        Print all customer info

        """
        return(
            f"\nCustomer: {self.username}\n"
            f"Checking: {self.checking}\n"
            f"Savings: {self.savings}\n"
            f"Credit: {self.credit}, Limit: ${self.credit.credit_limit}\n"
        )
    

In [ ]:
def load_customers(file_path):
    """
    Read customer data from a tab-separated CSV file
    and return a list of Customer objects.
    """
    customers = []

    with open(file_path, "r") as file:
        first_line = file.readline().strip()

        headers = first_line.split(",")

        reader = csv.DictReader(file, fieldnames=headers)

        for row in reader:

            username = row["username"]

            checking = Checking(row["checking_id"], float(row["checking_balance"]))
            savings = Savings(row["savings_id"], float(row["savings_balance"]))
            credit = Credit(
                row["credit_id"],
                float(row["credit_balance"]),
                float(row["credit_limit"])
            )

            customer = Customer(username, checking, savings, credit)
            customers.append(customer)

    return customers



In [ ]:
# Interface
def interface():
    """
    Runs the menu for the banking system

    """
    data_file = input("Enter file address: ")
    customers = load_customers(data_file)

    while True:

        print(
            f"\n----Bank Menu----\n"
            f"1. View Customers \n"
            f"2. Deposit \n"
            f"3. Withdraw \n"
            f"4. Credit Charge \n"
            f"5. Credit Payment \n"
            f"6. Exit", flush = True
        )
        
        choice = input("Enter choice: ")

        if choice == "1":
            for cust in customers:
                print(cust)

        elif choice == "2":
            user = input("Username: ")
            acct_type = input("Checkings or Savings?: ").lower()
            amount = float(input("Amount: "))
            
            if acct_type != "checkings" or acct_type != "savings":
                print("Invalid Input")
                break

            for c in customers:
                if c.username == user:
                    if acct_type == "checking":
                        c.checking.deposit(amount)
                    else:
                        c.savings.deposit(amount)
        
        elif choice == "3":
            user = input("Username: ")
            acct_type = input("Checkings or Savings?: ").lower()
            amount = float(input("Amount: "))

            if acct_type != "checkings" or acct_type != "savings":
                print("Invalid Input")
                break

            for c in customers:
                if c.usernmae == user:
                    if acct_type == "checking":
                        c.checking.withdraw(amount)
                    else:
                        c.savings.withdraw(amount)
        
        elif choice == "4":
            user = input("Username:" )
            amount = float(input("Charge Amount: "))

            for c in customers:
                if c.usernmae == user:
                    c.credit.charge(amount)

        elif choice == "5":
            user = input("Username: ")
            amount = float(input("Payment Amount: "))

            for c in customers:
                if c.username == user:
                    c.credit.make_payment(amount)
        
        elif choice == "6":
            print("Goodbye!")
            break
        else:
            print("Invalid Input")
            break


In [ ]:
interface()